In [6]:
using LowRankModels
using CSV
using DataFrames
using Pkg
using StatsBase
using Random, SparseArrays
#using DataArrays

In [8]:
imp = CSV.read("../data/Random_Forest_Importance.csv")
impp = CSV.read("../data/Positive_Random_Forest_Importance.csv")

hyper = CSV.read("../data/Hypertension_Matrix.csv")
hyper = select!(hyper, Not(:Column1))
hyper = select!(hyper, Not(:SUBJECT_ID )) 
hyper

,003,004,005,007,008,009,010,011,012,013,014,015
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
#column names

for name in names(hyper)
    print(":", name, ", ")
end

#describe(df3)

:003, :004, :005, :007, :008, :009, :010, :011, :012, :013, :014, :015, :018, :023, :027, :031, :032, :033, :034, :035, :036, :038, :039, :040, :041, :042, :046, :047, :048, :049, :052, :053, :054, :057, :058, :070, :075, :077, :078, :079, :088, :091, :093, :094, :097, :099, :110, :111, :112, :115, :117, :121, :122, :123, :127, :130, :131, :132, :133, :135, :136, :137, :138, :139, :140, :141, :142, :143, :144, :145, :146, :147, :148, :149, :150, :151, :152, :153, :154, :155, :156, :157, :158, :159, :160, :161, :162, :163, :164, :170, :171, :172, :173, :174, :175, :176, :179, :180, :182, :183, :184, :185, :187, :188, :189, :191, :192, :193, :194, :195, :196, :197, :198, :199, :200, :201, :202, :203, :204, :205, :206, :207, :208, :209, :210, :211, :212, :213, :214, :215, :216, :217, :218, :220, :221, :223, :225, :226, :227, :228, :229, :230, :232, :233, :235, :236, :237, :238, :239, :240, :241, :242, :244, :245, :246, :249, :250, :251, :252, :253, :254, :255, :256, :257, :258, :259, :260

In [5]:
#drop columns
#df3 = select!(df3, Not(:Column1))
df3 = CSV.read("../data/Diabetes_Numerical.csv")
df3 = select!(df3, Not(:SUBJECT_ID )) 
df3

,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government
,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,3.666,1,4,0,1,0,0,0
2,1.2885,1,6,0,1,0,0,0
3,1.0508,1,8,1,0,0,0,0
4,14.2664,2,9,0,1,0,0,0
5,0.5124,1,4,0,1,0,0,0
6,3.5466,1,6,0,1,0,0,0
7,1.1224,1,8,1,0,0,0,0
8,5.3757,1,7,1,0,0,0,0
9,1.1397,1,7,0,1,0,0,0


In [9]:
diabetesdrg = CSV.read("../data/diabetesdrg.csv")
diabetesdrg = select!(diabetesdrg, Not(:SUBJECT_ID )) 
diabetesdrg

,LOS,HOSPITALIZATION,ORDINAL_AGE,ELECTIVE,EMERGENCY,NEWBORN,URGENT,Government
,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,14.2664,2,9,0,1,0,0,0
2,5.3757,1,7,1,0,0,0,0
3,1.9138,1,7,0,1,0,0,0
4,2.2017,1,9,0,1,0,0,0
5,1.2469,1,8,1,0,0,0,0
6,5.1326,1,8,0,1,0,0,0
7,6.1813,2,9,0,1,0,0,0
8,3.7078,1,8,0,1,0,0,0
9,26.919,7,6,0,1,0,0,0


# hyper simple kmeans with init

In [15]:
k = 5
A= hyper
losses = QuadLoss() # minimize squared distance to cluster centroids
rx = UnitOneSparseConstraint() # each row is assigned to exactly one cluster
ry = ZeroReg() # no regularization on the cluster centroids
glrm = GLRM(A,losses,rx,ry,k) #, sparse_na=false

TypeError: TypeError: non-boolean (Missing) used in boolean context

In [12]:
init_kmeanspp!(glrm)

UndefVarError: UndefVarError: glrm not defined

# GLRM k-means loss function with positive weight hypertension